In [114]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, RandomRotation, RandomFlip

import pathlib

In [99]:
train_url = './data/train/'
valid_url = './data/valid/'
test_url = './data/test/'

train_data_dir = pathlib.Path(train_url).with_suffix('')
valid_data_dir = pathlib.Path(test_url).with_suffix('')
test_data_dir = pathlib.Path(valid_url).with_suffix('')

train_image_count = len(list(train_data_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_data_dir.glob('*/*.jpg')))
test_image_count = len(list(test_data_dir.glob('*/*.jpg')))

print(f'Počet trénovacích obrázků: {train_image_count}')
print(f'Počet validačních obrázků: {valid_image_count}')
print(f'Počet testovacích obrázků: {test_image_count}')

Počet trénovacích obrázků: 7978
Počet validačních obrázků: 285
Počet testovacích obrázků: 285


In [105]:
batch_size = 32
img_height = 64
img_width = 64

In [111]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    seed=0,
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    valid_data_dir,
    seed=0,
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Found 7978 files belonging to 57 classes.
Found 285 files belonging to 57 classes.


In [112]:
class_names = train_ds.class_names
print(class_names)

['air hockey', 'ampute football', 'archery', 'arm wrestling', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling', 'billiards', 'bmx', 'bowling', 'boxing', 'bull riding', 'canoe slamon', 'cheerleading', 'cricket', 'curling', 'disc golf', 'fencing', 'field hockey', 'figure skating', 'football', 'formula 1 racing', 'frisbee', 'giant slalom', 'golf', 'hammer throw', 'high jump', 'hockey', 'horse jumping', 'horse racing', 'hurdles', 'hydroplane racing', 'jousting', 'judo', 'lacrosse', 'motorcycle racing', 'nascar racing', 'pole dancing', 'polo', 'rock climbing', 'rugby', 'sailboat racing', 'ski jumping', 'skydiving', 'snow boarding', 'speed skating', 'sumo wrestling', 'surfing', 'swimming', 'table tennis', 'tennis', 'track bicycle', 'volleyball', 'water polo', 'weightlifting']


In [47]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [113]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

print(np.min(first_image), np.max(first_image))

0.0 1.0


Model

In [118]:
classes_len = len(class_names)
print(classes_len)
model = Sequential()
model.add(RandomFlip("vertical", input_shape=(img_height, img_width, 3)))
#model.add(RandomRotation(0.2, input_shape=(img_height, img_width, 3)))
model.add(Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(classes_len, activation = 'softmax'))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

57


In [116]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 224, 224, 3)      0         
 tion)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 54, 54, 64)      

In [120]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

Epoch 1/15


250/250 [==============================] - 316s 1s/step - loss: 4.0021 - accuracy: 0.0372 - val_loss: 3.9601 - val_accuracy: 0.0421
Epoch 2/15
250/250 [==============================] - 295s 1s/step - loss: 3.9794 - accuracy: 0.0402 - val_loss: 3.9781 - val_accuracy: 0.0316
Epoch 3/15
  7/250 [..............................] - ETA: 5:08 - loss: 3.9508 - accuracy: 0.0312

KeyboardInterrupt: 

In [97]:
from tensorflow.keras.preprocessing import image
for i in range(1, 6):
    img_url = './data/test/box/' + str(i) + '.jpg'
    img = image.load_img(img_url, target_size=(img_height, img_width))
    imgShow = image.load_img(img_url)

    # Convert the image to an array and expand dimensions
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])

    # Perform inference using the model
    val = model.predict(images, verbose=0)
    index = np.where(val == 1)[1]

    predicted_class_index = np.argmax(val)

    # Get the corresponding class label
    predicted_class_label = class_names[predicted_class_index]

    # Join class labels into a comma-separated string
    class_labels_string = ", ".join(class_names)

    # Print the predicted class label and the comma-separated list of all class labels
    print("Predicted class label:", predicted_class_label)
    # Display the vignette image
    plt.imshow(imgShow)
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './data/test/box/1.jpg'